En la rápida industria del comercio electrónico de hoy en día, la entrega rápida y eficiente de pedidos es crucial para el éxito empresarial. Para garantizar un cumplimiento de pedidos sin problemas, las empresas a menudo se asocian con compañías de mensajería para enviar sus productos a los clientes. Sin embargo, gestionar los cargos cobrados por estas compañías de mensajería puede ser difícil, especialmente cuando se trata de un alto volumen de pedidos. Es uno de los problemas en tiempo real que experimentan las empresas B2B cuando los cargos estimados para la misma factura no coinciden. En este artículo, te guiaré a través de una solución para un problema como este basado en el Análisis de Precisión de Cargos de Mensajería B2B utilizando Python.

#"Análisis de Precisión de Cargos de Mensajería B2B"

El análisis de precisión de cargos de mensajería B2B se centra en evaluar la exactitud de las tarifas cobradas por las compañías de mensajería en la entrega de bienes en transacciones B2B. El objetivo es garantizar que las empresas sean facturadas adecuadamente por los servicios proporcionados por las compañías de mensajería.

In [1]:
import pandas as pd

order_report = pd.read_csv('Order Report.csv')
sku_master = pd.read_csv('SKU Master.csv')
pincode_mapping = pd.read_csv('pincodes.csv')
courier_invoice = pd.read_csv('Invoice.csv')
courier_company_rates = pd.read_csv('Courier Company - Rates.csv')

print("Order Report:")
print(order_report.head())
print("\nSKU Master:")
print(sku_master.head())
print("\nPincode Mapping:")
print(pincode_mapping.head())
print("\nCourier Invoice:")
print(courier_invoice.head())
print("\nCourier Company rates:")
print(courier_company_rates.head())

Order Report:
   ExternOrderNo            SKU  Order Qty  Unnamed: 3  Unnamed: 4
0     2001827036  8904223818706        1.0         NaN         NaN
1     2001827036  8904223819093        1.0         NaN         NaN
2     2001827036  8904223819109        1.0         NaN         NaN
3     2001827036  8904223818430        1.0         NaN         NaN
4     2001827036  8904223819277        1.0         NaN         NaN

SKU Master:
             SKU  Weight (g)  Unnamed: 2  Unnamed: 3  Unnamed: 4
0  8904223815682         210         NaN         NaN         NaN
1  8904223815859         165         NaN         NaN         NaN
2  8904223815866         113         NaN         NaN         NaN
3  8904223815873          65         NaN         NaN         NaN
4  8904223816214         120         NaN         NaN         NaN

Pincode Mapping:
   Warehouse Pincode  Customer Pincode Zone  Unnamed: 3  Unnamed: 4
0             121003            507101    d         NaN         NaN
1             121003       

In [3]:
 # revisar si hay valores faltantes
print("\nvalores faltantes in Website Order Report:")
print(order_report.isnull().sum())
print("\nvalores faltantes in SKU Master:")
print(sku_master.isnull().sum())
print("\nvalores faltantes in Pincode Mapping:")
print(pincode_mapping.isnull().sum())
print("\nvalores faltantes in Courier Invoice:")
print(courier_invoice.isnull().sum())
print("\nvalores faltantes in courier company rates:")
print(courier_company_rates.isnull().sum())


valores faltantes in Website Order Report:
ExternOrderNo      0
SKU                0
Order Qty          0
Unnamed: 3       400
Unnamed: 4       400
dtype: int64

valores faltantes in SKU Master:
SKU            0
Weight (g)     0
Unnamed: 2    66
Unnamed: 3    66
Unnamed: 4    66
dtype: int64

valores faltantes in Pincode Mapping:
Warehouse Pincode      0
Customer Pincode       0
Zone                   0
Unnamed: 3           124
Unnamed: 4           124
dtype: int64

valores faltantes in Courier Invoice:
AWB Code                0
Order ID                0
Charged Weight          0
Warehouse Pincode       0
Customer Pincode        0
Zone                    0
Type of Shipment        0
Billing Amount (Rs.)    0
dtype: int64

valores faltantes in courier company rates:
fwd_a_fixed         0
fwd_a_additional    0
fwd_b_fixed         0
fwd_b_additional    0
fwd_c_fixed         0
fwd_c_additional    0
fwd_d_fixed         0
fwd_d_additional    0
fwd_e_fixed         0
fwd_e_additional    0
rto_

#Limpieza de datos

In [4]:
# eliminar columnas sin nombre en Website Order Report DataFrame
order_report = order_report.drop(columns=['Unnamed: 3', 'Unnamed: 4'])

# eliminar columnas sin nombre en SKU Master DataFrame
sku_master = sku_master.drop(columns=['Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4'])

# eliminar columnas sin nombre enPincode Mapping DataFrame
pincode_mapping = pincode_mapping.drop(columns=['Unnamed: 3', 'Unnamed: 4'])

#Ahora vamos a fusionar los conjuntos de datos del informe de pedidos y del maestro de SKU según la columna SKU común.

In [5]:
# unir  Order Report y SKU Master en base  al SKU
merged_data = pd.merge(order_report, sku_master, on='SKU')
print(merged_data.head())

   ExternOrderNo            SKU  Order Qty  Weight (g)
0     2001827036  8904223818706        1.0         127
1     2001821995  8904223818706        1.0         127
2     2001819252  8904223818706        1.0         127
3     2001816996  8904223818706        1.0         127
4     2001814580  8904223818706        1.0         127


#El 'ExternOrderNo' no es más que el ' Order Id' en otros datasets. por lo cual se  renombra esta columna:

In [6]:
# Renombrando la columna  "ExternOrderNo" a "Order ID" en el  merged_data DataFrame
merged_data = merged_data.rename(columns={'ExternOrderNo': 'Order ID'})

#Ahora se une el courier_invoice con el pincode_mapping dataset

Primero extraemos los códigos PIN únicos de los clientes del conjunto de datos de mapeo de códigos postales y creamos un nuevo DataFrame llamado "abc_courier" para almacenar esta información.

Luego seleccionamos columnas específicas (“Order ID”, “Customer Pincode”, “Type of Shipment”)  del conjunto de datos de la factura de mensajería y creamos un nuevo DataFrame llamado "courier_abc" para almacenar este subconjunto de datos.

Luego fusionamos el DataFrame 'courier_abc' con el DataFrame 'abc_courier' basado en la columna 'Customer Pincode’. Esta operación de fusión nos ayuda a asociar PIN codes de los clientes con sus respectivos pedidos y tipos de envío. El DataFrame resultante se llama 'pincodes'.

In [7]:
abc_courier = pincode_mapping.drop_duplicates(subset=['Customer Pincode'])
courier_abc= courier_invoice[['Order ID', 'Customer Pincode','Type of Shipment']]
pincodes= courier_abc.merge(abc_courier,on='Customer Pincode')
print(pincodes.head())

     Order ID  Customer Pincode Type of Shipment  Warehouse Pincode Zone
0  2001806232            507101  Forward charges             121003    d
1  2001806273            486886  Forward charges             121003    d
2  2001806408            532484  Forward charges             121003    d
3  2001806458            143001  Forward charges             121003    b
4  2001807012            515591  Forward charges             121003    d


#Se une pincodes con el dataframe principal

In [8]:
merged2 = merged_data.merge(pincodes, on='Order ID')

#calculo del peso en kgdividiendo la columna 'weight(g)' en 'merged2' dataframe por 100

In [9]:
merged2['Weights (Kgs)'] = merged2['Weight (g)'] / 1000

#ahora se calcula los rangos de peso 'weight slabs´

In [10]:
def weight_slab(weight):
    i = round(weight % 1, 1)
    if i == 0.0:
        return weight
    elif i > 0.5:
        return int(weight) + 1.0
    else:
        return int(weight) + 0.5

merged2['Weight Slab (KG)'] = merged2['Weights (Kgs)'].apply(weight_slab)
courier_invoice['Weight Slab Charged by Courier Company']=(courier_invoice['Charged Weight']).apply(weight_slab)

La función weight_slab() se define para determinar el rango de peso según el peso del envío. Toma el peso de entrada y aplica ciertas condiciones para calcular el rango de peso. Así es como funciona:

Primero, la función calcula el residuo del peso dividido por 1 y lo redondea a un decimal. Si el residuo es 0.0, significa que el peso es un múltiplo de 1 KG, y la función devuelve el peso tal como está.

Si el residuo es mayor que 0.5, significa que el peso excede el siguiente medio rango de KG. En este caso, la función redondea el peso al entero más cercano y le suma 1.0, lo que representa el próximo rango de peso más pesado.

Si el residuo es menor o igual a 0.5, significa que el peso cae en el rango actual de medio KG. En este caso, la función redondea el peso al entero más cercano y le suma 0.5, lo que representa el rango de peso actual.

Ahora se renombra las columnas para preparar el DataFrame deseado:

In [11]:
courier_invoice = courier_invoice.rename(columns={'Zone': 'Delivery Zone Charged by Courier Company'})
merged2 = merged2.rename(columns={'Zone': 'Delivery Zone As Per ABC'})
merged2 = merged2.rename(columns={'Weight Slab (KG)': 'Weight Slab As Per ABC'})

#ahora se calculan los costos esperados

En este código, recorremos cada fila del DataFrame 'merged2' para calcular los cargos esperados basados en las tarifas de ABC. Recuperamos las tasas y parámetros necesarios, como cargos fijos y recargos por rango de peso para envíos hacia adelante y RTO, según el área de entrega.

Luego, determinamos el rango de peso para cada fila. Si el tipo de envío es 'Cargos por Adelantado', calculamos el peso adicional más allá del rango de peso básico (0.5 KG) y aplicamos los cargos adicionales correspondientes. Para los envíos de "Cargos por Adelantado y RTO", consideramos cargos adicionales por componentes de término y RTO.

Finalmente, almacenamos los cargos esperados calculados en la columna "Cargos esperados según ABC" del DataFrame "merged2". Esto nos permite comparar los cargos esperados con los cargos facturados para analizar la precisión de los cargos de la empresa de mensajería.


In [12]:
total_expected_charge = []

for _, row in merged2.iterrows():
    fwd_category = 'fwd_' + row['Delivery Zone As Per ABC']
    fwd_fixed = courier_company_rates.at[0, fwd_category + '_fixed']
    fwd_additional = courier_company_rates.at[0, fwd_category + '_additional']
    rto_category = 'rto_' + row['Delivery Zone As Per ABC']
    rto_fixed = courier_company_rates.at[0, rto_category + '_fixed']
    rto_additional = courier_company_rates.at[0, rto_category + '_additional']

    weight_slab = row['Weight Slab As Per ABC']

    if row['Type of Shipment'] == 'Forward charges':
        additional_weight = max(0, (weight_slab - 0.5) / 0.5)
        total_expected_charge.append(fwd_fixed + additional_weight * fwd_additional)
    elif row['Type of Shipment'] == 'Forward and RTO charges':
        additional_weight = max(0, (weight_slab - 0.5) / 0.5)
        total_expected_charge.append(fwd_fixed + additional_weight * (fwd_additional + rto_additional))
    else:
        total_expected_charge.append(0)

merged2['Expected Charge as per ABC'] = total_expected_charge
print(merged2.head())

     Order ID            SKU  Order Qty  Weight (g)  Customer Pincode  \
0  2001827036  8904223818706        1.0         127            173213   
1  2001827036  8904223819093        1.0         150            173213   
2  2001827036  8904223819109        1.0         100            173213   
3  2001827036  8904223818430        1.0         165            173213   
4  2001827036  8904223819277        1.0         350            173213   

  Type of Shipment  Warehouse Pincode Delivery Zone As Per ABC  Weights (Kgs)  \
0  Forward charges             121003                        e          0.127   
1  Forward charges             121003                        e          0.150   
2  Forward charges             121003                        e          0.100   
3  Forward charges             121003                        e          0.165   
4  Forward charges             121003                        e          0.350   

   Weight Slab As Per ABC  Expected Charge as per ABC  
0                 

#Ahora se une con courier_invoice para mostrar el DataFrame final:

In [13]:
merged_output = merged2.merge(courier_invoice, on='Order ID')
print(merged_output.head())

     Order ID            SKU  Order Qty  Weight (g)  Customer Pincode_x  \
0  2001827036  8904223818706        1.0         127              173213   
1  2001827036  8904223819093        1.0         150              173213   
2  2001827036  8904223819109        1.0         100              173213   
3  2001827036  8904223818430        1.0         165              173213   
4  2001827036  8904223819277        1.0         350              173213   

  Type of Shipment_x  Warehouse Pincode_x Delivery Zone As Per ABC  \
0    Forward charges               121003                        e   
1    Forward charges               121003                        e   
2    Forward charges               121003                        e   
3    Forward charges               121003                        e   
4    Forward charges               121003                        e   

   Weights (Kgs)  Weight Slab As Per ABC  Expected Charge as per ABC  \
0          0.127                     0.5                

#ahora se calculan las diferencias en el cargo y cargo esperado para cada pedido

In [16]:
df_diff = merged_output
df_diff['Difference (Rs.)'] = df_diff['Billing Amount (Rs.)'] - df_diff['Expected Charge as per ABC']

df_new = df_diff[['Order ID', 'Difference (Rs.)', 'Expected Charge as per ABC']]

print(df_new.head())

     Order ID  Difference (Rs.)  Expected Charge as per ABC
0  2001827036              61.3                        56.6
1  2001827036              61.3                        56.6
2  2001827036              61.3                        56.6
3  2001827036              61.3                        56.6
4  2001827036              61.3                        56.6


#Ahora vamos a resumir la precisión de los cargos de mensajería B2B basados en los precios cobrados y los precios esperados:

In [17]:
# calculo del total de ordenes por categoria 
total_correctly_charged = len(df_new[df_new['Difference (Rs.)'] == 0])
total_overcharged = len(df_new[df_new['Difference (Rs.)'] > 0])
total_undercharged = len(df_new[df_new['Difference (Rs.)'] < 0])

# calculo del monto total en cada categoria  
amount_overcharged = abs(df_new[df_new['Difference (Rs.)'] > 0]['Difference (Rs.)'].sum())
amount_undercharged = df_new[df_new['Difference (Rs.)'] < 0]['Difference (Rs.)'].sum()
amount_correctly_charged = df_new[df_new['Difference (Rs.)'] == 0]['Expected Charge as per ABC'].sum()

# nuevo dataframe para el resumen 
summary_data = {'Description': ['Total Orders where ABC has been correctly charged',
                                'Total Orders where ABC has been overcharged',
                                'Total Orders where ABC has been undercharged'],
                'Count': [total_correctly_charged, total_overcharged, total_undercharged],
                'Amount (Rs.)': [amount_correctly_charged, amount_overcharged, amount_undercharged]}

df_summary = pd.DataFrame(summary_data)

print(df_summary)

                                         Description  Count  Amount (Rs.)
0  Total Orders where ABC has been correctly charged     12         507.6
1        Total Orders where ABC has been overcharged    382       33750.5
2       Total Orders where ABC has been undercharged      7        -165.2


#visualizacion de la proporcion de errores

In [18]:
import plotly.graph_objects as go
fig = go.Figure(data=go.Pie(labels=df_summary['Description'],
                            values=df_summary['Count'],
                            textinfo='label+percent',
                            hole=0.4))
fig.update_layout(title='Proporcion')

fig.show()

#Resumen

El análisis de precisión de cargos de mensajería B2B se centra en evaluar la exactitud de las tarifas cobradas por las compañías de mensajería para la entrega de bienes en transacciones B2B. En este tipo de problemas, nuestro objetivo es garantizar que las empresas sean facturadas adecuadamente por los servicios proporcionados por las compañías de mensajería. Espero que hayas disfrutado este artículo sobre el Análisis de Precisión de Cargos de Mensajería B2B utilizando Python. No dudes en hacer preguntas valiosas en la sección de comentarios a continuación.